In [ ]:
from mpmath import *
import math
mp.pretty = True


In [ ]:
def func_val(x):
    val = 0
    for i in range(len(S)):
        val += (S[i])*pow(x, 7-i)
    return val


In [ ]:
def poly_multi(p1, p2):
    m = len(p1)
    n = len(p2)
    c = [0]*(m+n-1)
    for i in range(m):
        for j in range(n):
            c[i+j] += p1[i]*p2[j]
    return c

def exponential(p1):
    c = [1]
    for i in range(5):
        c = poly_multi(c, p1)
    return c


In [ ]:
padding = "Cryptophile: This door has RSA encryption with exponent 5 and the password is"
n = 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
c = 49351673615347257945680235356974693522445051704268941376930191910678842821422559260707102208885603407978512689473338697184460712802031281127306417167492155362084054594227492464631583586721152629421050980558541147791359073864821324298547651163372720033106842194053857702267966212898166226127699424995214587480

In [ ]:
x = [8*x for x in range(1, 60)]
p = ""
for ch in padding:
    p += "0"*(8-len(str(bin(ord(ch)))[2:]))+str(bin(ord(ch)))[2:]


In [ ]:
for i in x:
    mp.prec = i
    K = 1 << i
    a = int(p, 2)*K
    Rx = exponential([K, a])
    Rx[-1] -= c
    R_poly = [[0 for i in range(7)] for i in range(7)]
    for i in range(5):
        R_poly[6-i][6-i] = n*pow(K, i)
    R_poly[1] = [0]+Rx
    R_poly[0] = [K*x for x in Rx]+[0]
    with open('Matrix.txt', 'w') as f:
        f.write(str(R_poly).replace(',', ' '))
    !fplll "Matrix.txt" > "vectors.txt"
    with open('vectors.txt', 'r') as f:
        L = f.read()
        L = L.replace('[', '')
        L = L.replace(']', '')
    L = L.splitlines()
    V = [x.split(' ') for x in L]
    T = [[int(y) for y in x if y != ''] for x in V]
    T = T[:-1]
    S = T[0]
    S = S[:7]
    y = findroot(func_val, (0.1, 1), tol=1/K, verify=False, solver='anderson')
    y = int(mpf(y*K))
    z = bin(y)
    z = z[2:]
    if len(z) % 8 != 0:
        z = '0'*(8-len(z) % 8)+z
    itr = len(z)//8
    ans = ""
    for i in range(itr):
        e = 128
        sum = 0
        for j in range(8):
            sum += e*(z[i*8+j] == '1')
            e //= 2
        ans += chr(sum)
    print(f"{ans}")